# Knowledge Graph

## 산업인공지능학과
## 2024254022
## 정현일



In [ ]:
import re
import pandas as pd
import bs4
import requests
import spacy

from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
from spacy.tokens import Span

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm


pd.set_option('display.max_colwidth', 200)
%matplotlib inline


candidate_sentences = pd.read_csv('wiki_sentences_v2.csv')
print(candidate_sentences.shape)
print(candidate_sentences)

In [ ]:
entiry_pairs = []

In [ ]:

def get_entities(sent):
    ent1 = ""
    ent2 = ""
    prv_tok_dep = ""
    prv_tok_text = ""
    prefix = ""
    modifier = ""
    for tok in nlp(sent):
        # 토큰이 구두점(punctuation mark)이면 다음 토큰으로 이동
        if tok.dep_ != "punct":
            if tok.dep_ == "compound": # 토큰이 복합어인 경우
                prefix = tok.text
                if prv_tok_dep == "compound": # 직전 토큰이 복합어이면 현재 토큰과 결합
                    prefix = prv_tok_text + " " + tok.text
            if tok.dep_.endswith("mod") == True: # 토큰이 수식어(modifier)인 경우
                modifier = tok.text
                if prv_tok_dep == "compound": # 직전 톸ㄴ이 수식어이면 현재 토큰을 결합
                    modifier = prv_tok_text + " " + tok.text
            
            if tok.dep_.find("subj") == True: # 주어(subject)인 경우,
                ent1 = modifier + " " + prefix + " " + tok.text # 수식어와 현재 토큰 결합 => 개체명 생성
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""
            if tok.dep_.find("obj") == True: # 목적어인 경우
                ent2 = modifier + " " + prefix + " " + tok.text # 수식어와 현재 토큰 결합 => 객채명 생성
                
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text
    return [ent1.strip(), ent2.strip()] # 식별된 개체명 반환




def get_relation(sent):
    doc = nlp(sent)
    matcher = Matcher(nlp.vocab)
    
    # 패턴 정의
    pattern = [{'DEP' : 'ROOT'},
               {'DEP' : 'prep', 'OP' : "?"},
               {'DEP' : 'agnet', 'OP' : "?"},
               {'POS' : 'ADJ', 'OP' : "?"}]

    matcher.add('matching_1', [pattern])
    
    matches = matcher(doc)
    print('matches : ', matches)
    k = len(matches) - 1 
    
    span = doc[matches[k][1]:matches[k][2]]

    return(span.text)